# 02 - Data Cleaning & Harmonization

This notebook cleans and harmonizes raw USITC trade data for analysis.

**Key Steps:**
1. Load raw USITC data files (wide format)
2. Transform from wide to long format
3. Standardize country names
4. Apply inflation adjustment (convert to real dollars)
5. Calculate derived metrics (shares, growth rates)
6. Save processed dataset

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from data_loader import save_processed_data, DATA_RAW, DATA_PROCESSED, DATA_REFERENCE
from classification_mapper import standardize_country_names, add_historical_period
from transformers import calculate_country_shares, calculate_yoy_growth, adjust_for_inflation

print("Modules loaded successfully")

Modules loaded successfully


## Step 1: Load Raw USITC Data Files

In [2]:
# Define file paths
usitc_dir = DATA_RAW / 'usitc'

imports_file = usitc_dir / 'imports_1995_2024.csv'
exports_file = usitc_dir / 'exports_1995_2024.csv'

print(f"Imports file exists: {imports_file.exists()}")
print(f"Exports file exists: {exports_file.exists()}")

Imports file exists: True
Exports file exists: True


In [3]:
# Load raw wide-format data
imports_wide = pd.read_csv(imports_file)
exports_wide = pd.read_csv(exports_file)

print(f"Imports: {imports_wide.shape[0]} countries, {imports_wide.shape[1]} columns")
print(f"Exports: {exports_wide.shape[0]} countries, {exports_wide.shape[1]} columns")

print("\nImports sample:")
display(imports_wide.head())

print("\nExports sample:")
display(exports_wide.head())

Imports: 239 countries, 31 columns
Exports: 243 countries, 31 columns

Imports sample:


,Country,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,0.01,0.02,0.01,0.02,0.01,0.00,0.00,0.00,0.06,...,0.02,0.03,0.01,0.03,0.04,0.02,0.02,0.02,0.02,0.02
1,Albania,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,...,0.16,0.12,0.04,0.07,0.06,0.05,0.07,0.14,0.08,0.13
2,Algeria,1.67,2.10,2.44,1.63,1.83,2.72,2.69,2.36,4.75,...,3.37,3.23,3.81,4.62,2.48,0.49,1.89,3.01,3.03,2.46
3,Andorra,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.05,0.00
4,Angola,2.24,2.69,2.78,2.25,2.42,3.56,3.10,3.11,4.26,...,2.81,2.86,2.60,2.70,0.95,0.47,1.04,1.60,1.16,1.87



Exports sample:


,Country,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,0.00,0.02,0.01,0.01,0.02,0.01,0.01,0.08,0.06,...,0.48,0.91,0.94,1.23,0.76,0.67,0.24,0.02,0.05,0.01
1,Albania,0.01,0.01,0.00,0.01,0.02,0.02,0.02,0.02,0.01,...,0.03,0.05,0.06,0.06,0.09,0.06,0.10,0.11,0.13,0.14
2,Algeria,0.77,0.63,0.69,0.65,0.46,0.87,1.05,0.98,0.49,...,1.88,2.19,1.06,1.26,1.00,0.73,0.78,1.20,1.20,1.01
3,Andorra,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
4,Angola,0.26,0.27,0.28,0.35,0.25,0.23,0.28,0.37,0.49,...,1.17,1.25,0.81,0.52,0.53,0.47,0.44,0.65,0.60,0.68


## Step 2: Transform from Wide to Long Format

In [4]:
def wide_to_long(df_wide: pd.DataFrame, trade_type: str) -> pd.DataFrame:
    """
    Transform USITC wide-format data to long format.
    
    Args:
        df_wide: DataFrame with Country column and year columns (1995, 1996, ...)
        trade_type: 'import' or 'export'
    
    Returns:
        Long-format DataFrame with columns: country, year, value, trade_type
    """
    # Get the country column name (first column)
    country_col = df_wide.columns[0]
    
    # Get year columns (all numeric column names)
    year_cols = [c for c in df_wide.columns[1:] if str(c).replace('.', '').isdigit()]
    
    # Filter to only country and year columns
    df_subset = df_wide[[country_col] + year_cols].copy()
    
    # Melt from wide to long
    df_long = df_subset.melt(
        id_vars=[country_col],
        var_name='year',
        value_name='value'
    )
    
    # Rename country column
    df_long = df_long.rename(columns={country_col: 'country'})
    
    # Add trade type
    df_long['trade_type'] = trade_type
    
    # Convert year to integer
    df_long['year'] = df_long['year'].astype(int)
    
    # Convert value to numeric (handle any formatting issues)
    df_long['value'] = pd.to_numeric(
        df_long['value'].astype(str).str.replace(',', '').str.replace('"', ''),
        errors='coerce'
    )
    
    # Filter out Total row and other non-country rows
    exclude_rows = ['Total:', 'Total', 'Unspecified', 'Transshipment', 'Internat Organization']
    df_long = df_long[~df_long['country'].str.strip().isin(exclude_rows)]
    
    # Convert from billions to actual USD
    df_long['value'] = df_long['value'] * 1e9
    
    return df_long

# Transform both datasets
imports_long = wide_to_long(imports_wide, 'import')
exports_long = wide_to_long(exports_wide, 'export')

print(f"Imports (long): {len(imports_long):,} rows")
print(f"Exports (long): {len(exports_long):,} rows")

print("\nImports sample:")
display(imports_long.head(10))

Imports (long): 7,140 rows
Exports (long): 7,170 rows

Imports sample:


,country,year,value,trade_type
0,Afghanistan,1995,1.000000e+07,import
1,Albania,1995,1.000000e+07,import
2,Algeria,1995,1.670000e+09,import
3,Andorra,1995,0.000000e+00,import
4,Angola,1995,2.240000e+09,import
5,Anguilla,1995,0.000000e+00,import
6,Antigua and Barbuda,1995,0.000000e+00,import
7,Argentina,1995,1.760000e+09,import
8,Armenia,1995,2.000000e+07,import
9,Aruba,1995,4.200000e+08,import


In [5]:
# Combine imports and exports into single DataFrame
trade_df = pd.concat([imports_long, exports_long], ignore_index=True)

print(f"Combined dataset: {len(trade_df):,} rows")
print(f"\nTrade type distribution:")
print(trade_df['trade_type'].value_counts())

print(f"\nYear range: {trade_df['year'].min()} - {trade_df['year'].max()}")
print(f"Unique countries: {trade_df['country'].nunique()}")

Combined dataset: 14,310 rows

Trade type distribution:
trade_type
export    7170
import    7140
Name: count, dtype: int64

Year range: 1995 - 2024
Unique countries: 239


## Step 3: Standardize Country Names

In [6]:
# Apply country name standardization
trade_df = standardize_country_names(trade_df, country_col='country')

print(f"Unique countries after standardization: {trade_df['country'].nunique()}")

# Show top countries by total trade value
top_countries = trade_df.groupby('country')['value'].sum().sort_values(ascending=False).head(20)
print("\nTop 20 trading partners (total trade 1995-2024):")
for i, (country, value) in enumerate(top_countries.items(), 1):
    print(f"  {i:2}. {country}: ${value/1e12:.2f} trillion")

Unique countries after standardization: 239

Top 20 trading partners (total trade 1995-2024):
   1. Canada: $15.68 trillion
   2. Mexico: $12.46 trillion
   3. China: $11.70 trillion
   4. Japan: $5.91 trillion
   5. Germany: $4.20 trillion
   6. United Kingdom: $3.01 trillion
   7. South Korea: $2.91 trillion
   8. Taiwan: $2.13 trillion
   9. France: $2.00 trillion
  10. Netherlands: $1.68 trillion
  11. Italy: $1.62 trillion
  12. Brazil: $1.60 trillion
  13. India: $1.58 trillion
  14. Singapore: $1.42 trillion
  15. Ireland: $1.39 trillion
  16. Malaysia: $1.33 trillion
  17. Switzerland: $1.33 trillion
  18. Belgium: $1.20 trillion
  19. Vietnam: $1.12 trillion
  20. Thailand: $1.08 trillion


## Step 4: Apply Inflation Adjustment

In [7]:
# Load GDP deflator
deflator_df = pd.read_csv(DATA_REFERENCE / 'gdp_deflator.csv')
print(f"GDP Deflator loaded: {len(deflator_df)} years")
display(deflator_df.tail(10))

GDP Deflator loaded: 37 years


,year,deflator,notes
27,2016,96.951,Index 2017=100
28,2017,100.000,Index 2017=100 (base year)
29,2018,102.386,Index 2017=100
30,2019,104.028,Index 2017=100
31,2020,105.380,Index 2017=100
32,2021,109.528,Index 2017=100
33,2022,116.603,Index 2017=100
34,2023,120.477,Index 2017=100
35,2024,123.500,Index 2017=100 (estimate)
36,2025,126.500,Index 2017=100 (estimate)


In [8]:
# Apply inflation adjustment (base year 2020)
trade_df = adjust_for_inflation(
    trade_df,
    deflator_df,
    value_col='value',
    year_col='year',
    base_year=2020
)

print("Sample with real values:")
sample = trade_df[trade_df['country'] == 'China'].sort_values('year')
display(sample[['year', 'country', 'trade_type', 'value', 'value_real']].head(10))

Sample with real values:


,year,country,trade_type,value,value_real
41,1995,China,import,4.556000e+10,7.066695e+10
7181,1995,China,export,1.175000e+10,1.822513e+10
7420,1996,China,export,1.198000e+10,1.826227e+10
279,1996,China,import,5.150000e+10,7.850642e+10
517,1997,China,import,6.255000e+10,9.387221e+10
7659,1997,China,export,1.281000e+10,1.922467e+10
7898,1998,China,export,1.426000e+10,2.120986e+10
755,1998,China,import,7.116000e+10,1.058411e+11
993,1999,China,import,8.179000e+10,1.200556e+11
8137,1999,China,export,1.312000e+10,1.925821e+10


## Step 5: Calculate Derived Metrics

In [9]:
# Calculate country shares (% of total trade by year and trade type)
trade_df = calculate_country_shares(
    trade_df,
    value_col='value_real',
    country_col='country',
    year_col='year',
    trade_type_col='trade_type'
)

print("Top import sources in 2024 (by share):")
imports_2024 = trade_df[(trade_df['year'] == 2024) & (trade_df['trade_type'] == 'import')]
imports_2024_top = imports_2024.nlargest(10, 'share_pct')[['country', 'value_real', 'share_pct']]
display(imports_2024_top)

Top import sources in 2024 (by share):


,country,value_real,share_pct
7035,Mexico,4.313498e+11,15.476697
6943,China,3.743678e+11,13.432201
6938,Canada,3.514572e+11,12.610177
6981,Germany,1.368489e+11,4.910098
7006,Japan,1.266011e+11,4.542407
7133,Vietnam,1.164726e+11,4.179002
7098,South Korea,1.122489e+11,4.027456
7110,Taiwan,9.920226e+10,3.559346
7002,Ireland,8.812669e+10,3.161959
6998,India,7.452542e+10,2.673949


In [10]:
# Calculate year-over-year growth rates
trade_df = calculate_yoy_growth(
    trade_df,
    value_col='value_real',
    country_col='country',
    year_col='year',
    trade_type_col='trade_type'
)

print("China import growth over time:")
china_imports = trade_df[(trade_df['country'] == 'China') & (trade_df['trade_type'] == 'import')].sort_values('year')
display(china_imports[['year', 'value_real', 'share_pct', 'yoy_growth_pct']].tail(10))

China import growth over time:


,year,value_real,share_pct,yoy_growth_pct
4801,2015,5.307997e+11,21.487493,2.073375
5039,2016,5.026232e+11,21.147030,-5.308317
5277,2017,5.323481e+11,21.592337,5.913967
5515,2018,5.542573e+11,21.233533,4.115561
5753,2019,4.549469e+11,18.025036,-17.917745
5991,2020,4.325500e+11,18.553071,-4.922961
6229,2021,4.851532e+11,17.827281,12.161195
6467,2022,4.846542e+11,16.553127,-0.102866
6705,2023,3.737112e+11,13.886316,-22.891164
6943,2024,3.743678e+11,13.432201,0.175691


In [11]:
# Add historical period classification
trade_df = add_historical_period(trade_df, year_col='year')

print("Historical periods in data:")
print(trade_df.groupby('period')['year'].agg(['min', 'max', 'count']))

Historical periods in data:
                     min   max  count
period                               
COVID Era           2020  2021    954
China WTO Boom      2001  2007   3339
Financial Crisis    2008  2009    954
Post-COVID          2022  2024   1431
Post-Crisis Growth  2010  2017   3816
Pre-WTO China       1995  2000   2862
Trade War           2018  2019    954


In [12]:
# Final columns
print("Final columns:")
print(trade_df.columns.tolist())

print(f"\nFinal dataset shape: {trade_df.shape}")
print(f"Memory usage: {trade_df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

Final columns:
['country', 'year', 'value', 'trade_type', 'value_real', 'share', 'share_pct', 'yoy_growth', 'yoy_growth_pct', 'period']

Final dataset shape: (14310, 10)
Memory usage: 3.5 MB


## Step 6: Data Validation

In [13]:
# Check for missing values
print("Missing values:")
print(trade_df.isnull().sum())

# Check for negative values
print(f"\nNegative trade values: {(trade_df['value'] < 0).sum()}")

# Check year coverage
print(f"\nYears covered: {sorted(trade_df['year'].unique())}")

Missing values:
country              0
year                 0
value                0
trade_type           0
value_real           0
share                0
share_pct            0
yoy_growth        2997
yoy_growth_pct    2997
period               0
dtype: int64

Negative trade values: 0

Years covered: [np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


In [14]:
# Verify total trade roughly matches known values
print("Total US imports by year (billions USD, nominal):")
annual_imports = trade_df[trade_df['trade_type'] == 'import'].groupby('year')['value'].sum() / 1e9
print(annual_imports.tail(10).round(1))

Total US imports by year (billions USD, nominal):
year
2015    2248.8
2016    2186.7
2017    2339.6
2018    2536.1
2019    2491.6
2020    2331.4
2021    2828.5
2022    3239.7
2023    3076.8
2024    3266.3
Name: value, dtype: float64


## Step 7: Save Processed Data

In [15]:
# Ensure output directory exists
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Save combined processed data
output_file = DATA_PROCESSED / 'trade_data_1995_2024.csv'
trade_df.to_csv(output_file, index=False)
print(f"Saved processed data to: {output_file}")
print(f"File size: {output_file.stat().st_size / 1e6:.1f} MB")

# Also save separate imports and exports files
imports_processed = trade_df[trade_df['trade_type'] == 'import']
exports_processed = trade_df[trade_df['trade_type'] == 'export']

imports_processed.to_csv(DATA_PROCESSED / 'imports_processed.csv', index=False)
exports_processed.to_csv(DATA_PROCESSED / 'exports_processed.csv', index=False)

print(f"\nSaved {len(imports_processed):,} import records")
print(f"Saved {len(exports_processed):,} export records")

Saved processed data to: C:\Users\imste\Documents\Data_Trade-Import-Export\data\processed\trade_data_1995_2024.csv
File size: 1.9 MB



Saved 7,140 import records
Saved 7,170 export records


## Summary

In [16]:
print("=" * 60)
print("DATA PROCESSING COMPLETE")
print("=" * 60)
print(f"\nTotal records: {len(trade_df):,}")
print(f"Years: {trade_df['year'].min()} - {trade_df['year'].max()}")
print(f"Countries: {trade_df['country'].nunique()}")
print(f"Trade types: {trade_df['trade_type'].unique().tolist()}")
print(f"\nOutput files:")
print(f"  - {DATA_PROCESSED / 'trade_data_1995_2024.csv'}")
print(f"  - {DATA_PROCESSED / 'imports_processed.csv'}")
print(f"  - {DATA_PROCESSED / 'exports_processed.csv'}")
print("\n-> Proceed to 03_exploratory_analysis.ipynb")

DATA PROCESSING COMPLETE

Total records: 14,310
Years: 1995 - 2024
Countries: 239
Trade types: ['import', 'export']

Output files:
  - C:\Users\imste\Documents\Data_Trade-Import-Export\data\processed\trade_data_1995_2024.csv
  - C:\Users\imste\Documents\Data_Trade-Import-Export\data\processed\imports_processed.csv
  - C:\Users\imste\Documents\Data_Trade-Import-Export\data\processed\exports_processed.csv

-> Proceed to 03_exploratory_analysis.ipynb


# End of notebook